# Problem 2: Decoding

* We have an HMM and we know the transition probabilities and observation likelihoods
* Given an observation sequence, estimate the most likely hidden state sequence

For any model that contains hidden variables, the task of determining which sequence of variable is the underlying source of some sequence of observations is called the **decoding** task.


So in our example:


Given a sequence of ice-cream observations *3 1 3* an an HMM, find the best hidden weather sequence (temp, temp, temp) 



This is easy with brute force: just check each possible sequence.

# Brute Force

In [1]:
from fractions import Fraction
from markov import hot, cold, Temperature, STATES,OBSERVATIONS
from itertools import product
from algorithms import (
    p_observations_given_states,
    p_states,
    argmax,
    bestpath,
    brute_force_most_likely_sequence,
    backpointers,
)
from algorithms import virterbi
from coefficients import PI, A, B

In [2]:
sum([p_states(PI, A, list(sequence)) for sequence in product(STATES, repeat=3)])

Fraction(1, 1)

In [3]:
sum([
    p_states(PI,A,states) * p_observations_given_states(B,states,observations)
    for states in product(STATES, repeat=3)
    for observations in product(OBSERVATIONS,repeat=3)
])

Fraction(1, 1)

In [7]:
observations = [3,1,3]
print(f""" 
    Given the observations {observations},  the mostly likely
    sequence of hidden states is {brute_force_most_likely_sequence(PI,A,B,observations)}
""")


 
    Given the observations [3, 1, 3],  the mostly likely
    sequence of hidden states is [hot, cold, hot]



Which is what we would expect. 'Ice cream consumption goes up on hot days'

#### Cost

Again, this is comptutationally expensive.

Instead we use the 'Virterbi Algorithm'

# Virterbi Algorithm

> The idea is to process the observation sequence left to right, filling out the trellis.

> Each cell of the trellis, $v_t(j)$ represents the probability that the HMM is in state $j$ after seeing the first $t$ observations and passing through the most probable state sequence $q_1,...,q_{t-1}$, given the automaton $\lambda$


We compute values for each cell $v_t(j)$ by recursively taking the most probable path that could lead us to this cell.

$$ 

    v_t(j) = \max _{q_1,..., q_{t-1}} P(
        q_1...q_{t-1},o_1, o_2...o_t , q_t = j | \lambda)
$$

Again, this can be done recursively


$$
    v_t(j) = \max_{i=1}^N v_{t-1}(i) a_{ij} b_j (o_t)
$$

So the Virterbi probability at a time $t$ is a function of:

* $v_{t-1}(i)$ - the virtebi value in a previous cell
* $a_{ij}$ - the transition probability from that cell to this one
* $b_j{o_t} $- the likelihood of observing symbol o_t given the current state $j$

> Note that the Virterbi algorithm is identical to the forward algorithm except that it takes `max` over previous path probability rather than `sum`.


In [21]:
vs = list(virterbi(PI, A, B, observations))

In [22]:
[{key: float(value) for key, value in v.items()} for v in vs]

[{cold: 0.02, hot: 0.32},
 {cold: 0.064, hot: 0.0384},
 {cold: 0.0032, hot: 0.0128}]

Which agrees with the example given in the text.

## Backpointers

Armed with this we can compute a most-likely path through the states.

$$

    bt_t(j) = \argmax_{i=1}^N v_{t-1} (i) a_{ij} b_j(o_t); 1 \le j \le N , 1 \lt t \lt T
$$

Create a path probability matrix $viterbi[N,T]$


**for** each state `s` from 1 to N:

$verterbi[s1,1]  \leftarrow \pi_s * b_s(o_1)$
 
$backpointer[s,1] \leftarrow 0 $ 
 

**for** each time step t from 2 to T:
  *  **for** each state s from 1 to N:
        

        $viterbi[s,t]\leftarrow \max _{s'=1}^N viterbi[s',t-1] * a_{s',s} * b_s(o_t)$

        $backpointer[s,t] \leftarrow \argmax_{s'=1}^N viterbi[s',t-1] * a_{s',s} * b_(o_t)$  
  

$bestpathprob \leftarrow \max_{s=1}^N viterbi[s,T]$


$bestpathpointer \leftarrow\argmax_{s=1}^N virterbi[s,T]$
 

In [23]:
vs = list(virterbi(PI, A, B, observations))
vs

[{cold: Fraction(1, 50), hot: Fraction(8, 25)},
 {cold: Fraction(8, 125), hot: Fraction(24, 625)},
 {cold: Fraction(2, 625), hot: Fraction(8, 625)}]

In [24]:
list(backpointers(vs, A, B, observations))

[{cold: None, hot: None}, {cold: hot, hot: hot}, {cold: cold, hot: cold}]

In [9]:
bestpath(PI, A, B, observations)

[hot, cold, hot]

Which agrees with the brute force method.

### 
From the Appendix:

$bestpathpointer$  is the state which has the highest V at time T 

$bestpathprob$ is the value of V[T,S]

and 

$bestpath$ is the path that starts at $bestpathpointer$ and follows backpointer backwards

## Check Virtebi algorithm against brute force method

Now let's test it with other observations

In [10]:
observations2 = [3,1,3,2,3,1,2,1,1,2]

In [12]:
%%time
brute_force_most_likely_sequence(PI,A,B,observations2)

CPU times: user 15.2 ms, sys: 0 ns, total: 15.2 ms
Wall time: 15 ms


[hot, cold, hot, hot, hot, cold, cold, cold, cold, cold]

In [13]:
%%time
bestpath(PI, A, B, observations2)

CPU times: user 273 μs, sys: 40 μs, total: 313 μs
Wall time: 315 μs


[hot, cold, hot, hot, hot, cold, cold, cold, cold, cold]

In [16]:
checked=0
for length in range(1,8):

    for observations in product(OBSERVATIONS,repeat =length):
        a = bestpath(PI,A,B,observations)
        b = brute_force_most_likely_sequence(PI,A,B,observations)
        assert a==b
        checked+=1
checked

3279

3279